# Project 2: Difference Equations and Impulse Responses
This project examines linear-time invariant systems and their representations with diffeence equations, impulse responses and frequency responses.

## Objective:
1. We have the difference equation $$ y[n]-1.1y[n-1]+0.46y[n-2]=x[n]+0.04x[n-1]+0.76x[n-2]$$ which can be implemented using the <code>filter</code> command. Find the vectors <code>a</code> and <code>b</code> that satisfy this equation.


2. Create an impulse of length 40.
    1. Characterize the LTI system by finding the first 40 points of the impulse response.
    2. Plot the impulse response with <code>stem</code>.


3. Examine two ways of implementing an LTI system:
    1. Create a signal consisting of a 5--point unit height pulse followed by five zeros followed by another 5--point unit height pulse and then zeropadded to a total length of 40.
    2. Find the response of the system to an input signal using <code>conv</code> and <code>filter</code>.
    3. Are there differences? Why?


4. Examine the frequency response:
    1. Find an expression for the frequency response of the system described by the difference equation.
    2. Plot the magnitude and phase response of the system.
    3. What type of filter does this resemble?


5. Examine the response to two sine waves:
    1. Create two signals $x_{1}[n]=\cos(0.1\pi n)$ and $x_{2}[n]=\sin(0.4\pi n)$ both of length 100.
    2. Filter each signal with the filter in step 1.
    3. Examine the frequency response of the system.
    4. Add the two signals and filter the sum of the two. Which does the result resemble more, $x_{1}[n]$ or $x_{2}[n]$?